In [1]:
import os
import sys
currentdir = os.path.dirname(os.path.realpath('__file__'))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)
print(currentdir, parentdir)

/Users/owl/Projects/normalization/notebooks /Users/owl/Projects/normalization


In [2]:
import yaml
import requests
from norm import *

In [3]:
content = requests.get('https://raw.githubusercontent.com/108michael/congress-legislators/master/legislators-historical.yaml').content
source = yaml.load(content)

In [198]:
df = pd.DataFrame(source)

# test columns
#df['dict'] = [{'a': 1}] * len(df)
#df['list_obj'] = [['a'] * 2] * len(df)
#df['list_dict'] = [[{'a':10, 'b': 'hello'}] * 2] * len(df)
#df['list_list'] = [[[10]] * 2] * len(df)
#df['list_list_list_dict'] = [[[[{'a':10}],]] * 2] * len(df)

# sample group
df_family = df[~df.family.isna()]
df_family.head()

,bio,family,id,leadership_roles,name,other_names,terms
11737,"{'birthday': '1962-03-30', 'gender': 'M'}","[{'name': 'Nicholas J. Begich', 'relation': 's...","{'bioguide': 'B001265', 'thomas': '01898', 'li...",NaN,"{'first': 'Mark', 'last': 'Begich', 'official_...",NaN,"[{'type': 'sen', 'start': '2009-01-06', 'end':..."
11739,"{'birthday': '1953-05-26', 'gender': 'F'}","[{'name': 'Lawton M. Chiles Jr.', 'relation': ...","{'bioguide': 'H001049', 'thomas': '01902', 'li...",NaN,"{'first': 'Kay', 'last': 'Hagan', 'official_fu...",NaN,"[{'type': 'sen', 'start': '2009-01-06', 'end':..."
11743,"{'birthday': '1934-06-28', 'gender': 'M', 'rel...","[{'name': 'Sander Martin Levin', 'relation': '...","{'bioguide': 'L000261', 'thomas': '01384', 'li...",NaN,"{'first': 'Carl', 'last': 'Levin', 'official_f...",NaN,"[{'type': 'sen', 'start': '1979-01-15', 'end':..."
11744,"{'birthday': '1963-01-10', 'gender': 'M', 'rel...","[{'name': 'David H. Pryor', 'relation': 'son'}]","{'bioguide': 'P000590', 'thomas': '01701', 'li...",NaN,"{'first': 'Mark', 'last': 'Pryor', 'official_f...",NaN,"[{'type': 'sen', 'start': '2003-01-07', 'end':..."
11745,"{'birthday': '1937-06-18', 'gender': 'M', 'rel...","[{'name': 'Nelson Aldrich Rockefeller', 'relat...","{'bioguide': 'R000361', 'thomas': '01424', 'li...",NaN,"{'first': 'John', 'middle': 'D.', 'last': 'Roc...",NaN,"[{'type': 'sen', 'start': '1985-01-03', 'end':..."


In [199]:
ds = df_family.id.copy()
get_column_dict(ds)

,ballotpedia,bioguide,cspan,fec,govtrack,house_history,icpsr,lis,maplight,opensecrets,thomas,votesmart,washington_post,wikipedia
11737,Mark Begich,B001265,1030732,[S8AK00090],412326,NaN,40900,S319,808,N00029901,01898,79426,gIQAlYCM9O,Mark Begich
11739,Kay Hagan,H001049,1028015,[S8NC00239],412324,15594.0,40907,S320,806,N00029617,01902,21082,gIQA2GXW9O,Kay Hagan
11743,Carl Levin,L000261,2592,[S8MI00158],300066,NaN,14709,S131,587,N00001691,01384,53307,gIQA5NHW9O,Carl Levin
11744,Mark Pryor,P000590,1002201,[S0AR00028],300080,NaN,40301,S295,599,N00013823,01701,35,gIQAHnsz9O,Mark Pryor
11745,Jay Rockefeller,R000361,1238,[S4WV00027],300084,NaN,14922,S176,603,N00001685,01424,53360,gIQAElxz9O,Jay Rockefeller
11746,Mark Udall,U000038,7634,"[H8CO02087, S8CO00172]",400412,20877.0,29906,S325,497,N00008051,01595,12329,gIQAz6eM9O,Mark Udall
11757,John D. Dingell,D000355,470,[H6MI16034],400110,12268.0,2605,NaN,223,N00001783,00299,26917,gIQAUVCT9O,John Dingell
11765,NaN,H001032,57967,[H6NJ12144],400184,15560.0,29923,NaN,289,N00000860,01580,22574,gIQAJMR0DP,"Rush D. Holt, Jr."


In [200]:
result = []

# get all series
series_list = list(df_family.iteritems())
print(f'series_list length:: current: {len(series_list)}')

# iterate through each column/ series
for _name, seri in series_list:
    # type of the object of series' values
    inside = seri.apply(lambda x: type(x)).iloc[0]

    # case if it is list -- expand rowwise
    if inside == list:
        temp = get_obj(seri)
        print('expanding:: ', _name, inside)
        assert type(temp) == pd.Series
        series_list.append((_name, temp))

    # case if it is dict -- expand columnwise
    elif inside == dict:
        temp = get_column_dict(seri)
        print('normalizing:: ', _name, inside, f'normalized: {(temp.shape)}')
        if len(temp) > 1:
            assert type(temp) == pd.DataFrame
            series_list.extend(list(temp.iteritems()))
        else:
            assert type(temp) == pd.Series
            series_list.append(temp)

    # case of all others
    else:
        temp = get_column_obj(seri)
        assert type(temp) == pd.DataFrame
        result.append(temp)
        print('columns added: ', _name, inside, f'len: {temp.shape}')

print('exports :: ', len(result))
print([len(i) for i in result])

series_list length:: current: 7
normalizing::  bio <class 'dict'> normalized: (8, 3)
<Trial #  type: <class 'dict'> , len: 13>
expanding::  family <class 'list'>
normalizing::  id <class 'dict'> normalized: (8, 14)
columns added:  leadership_roles <class 'float'> len: (8, 2)
normalizing::  name <class 'dict'> normalized: (8, 6)
columns added:  other_names <class 'float'> len: (8, 2)
<Trial #  type: <class 'dict'> , len: 59>
expanding::  terms <class 'list'>
columns added:  birthday <class 'str'> len: (8, 2)
columns added:  gender <class 'str'> len: (8, 2)
columns added:  religion <class 'float'> len: (8, 2)
normalizing::  family <class 'dict'> normalized: (13, 2)
columns added:  ballotpedia <class 'str'> len: (8, 2)
columns added:  bioguide <class 'str'> len: (8, 2)
columns added:  cspan <class 'int'> len: (8, 2)
<Trial #  type: <class 'str'> , len: 9>
expanding::  fec <class 'list'>
columns added:  govtrack <class 'int'> len: (8, 2)
columns added:  house_history <class 'float'> len: (

In [206]:
for i in result:
    print(i)
    print()

   index leadership_roles
0  11737              NaN
1  11739              NaN
2  11743              NaN
3  11744              NaN
4  11745              NaN
5  11746              NaN
6  11757              NaN
7  11765              NaN

   index other_names
0  11737         NaN
1  11739         NaN
2  11743         NaN
3  11744         NaN
4  11745         NaN
5  11746         NaN
6  11757         NaN
7  11765         NaN

   index    birthday
0  11737  1962-03-30
1  11739  1953-05-26
2  11743  1934-06-28
3  11744  1963-01-10
4  11745  1937-06-18
5  11746  1950-07-18
6  11757  1926-07-08
7  11765  1948-10-15

   index gender
0  11737      M
1  11739      F
2  11743      M
3  11744      M
4  11745      M
5  11746      M
6  11757      M
7  11765      M

   index        religion
0  11737             NaN
1  11739             NaN
2  11743          Jewish
3  11744         Unknown
4  11745    Presbyterian
5  11746             NaN
6  11757  Roman Catholic
7  11765      Protestant

   index      

In [207]:
reduce(lambda x, y: (pd.merge(x, y)), result[0:25])

,index,leadership_roles,other_names,birthday,gender,religion,ballotpedia,bioguide,cspan,govtrack,...,votesmart,washington_post,wikipedia,first,last,middle,nickname,official_full,suffix,name
0,11737,NaN,NaN,1962-03-30,M,NaN,Mark Begich,B001265,1030732,412326,...,79426,gIQAlYCM9O,Mark Begich,Mark,Begich,NaN,NaN,Mark Begich,NaN,Nicholas J. Begich
1,11739,NaN,NaN,1953-05-26,F,NaN,Kay Hagan,H001049,1028015,412324,...,21082,gIQA2GXW9O,Kay Hagan,Kay,Hagan,NaN,NaN,Kay R. Hagan,NaN,Lawton M. Chiles Jr.
2,11743,NaN,NaN,1934-06-28,M,Jewish,Carl Levin,L000261,2592,300066,...,53307,gIQA5NHW9O,Carl Levin,Carl,Levin,NaN,NaN,Carl Levin,NaN,Sander Martin Levin
3,11744,NaN,NaN,1963-01-10,M,Unknown,Mark Pryor,P000590,1002201,300080,...,35,gIQAHnsz9O,Mark Pryor,Mark,Pryor,NaN,NaN,Mark L. Pryor,NaN,David H. Pryor
4,11745,NaN,NaN,1937-06-18,M,Presbyterian,Jay Rockefeller,R000361,1238,300084,...,53360,gIQAElxz9O,Jay Rockefeller,John,Rockefeller,D.,Jay,"John D. Rockefeller, IV",IV,Nelson Aldrich Rockefeller
5,11745,NaN,NaN,1937-06-18,M,Presbyterian,Jay Rockefeller,R000361,1238,300084,...,53360,gIQAElxz9O,Jay Rockefeller,John,Rockefeller,D.,Jay,"John D. Rockefeller, IV",IV,Nelson Aldrich
6,11745,NaN,NaN,1937-06-18,M,Presbyterian,Jay Rockefeller,R000361,1238,300084,...,53360,gIQAElxz9O,Jay Rockefeller,John,Rockefeller,D.,Jay,"John D. Rockefeller, IV",IV,Charles Harting Percy
7,11746,NaN,NaN,1950-07-18,M,NaN,Mark Udall,U000038,7634,400412,...,12329,gIQAz6eM9O,Mark Udall,Mark,Udall,E.,NaN,Mark Udall,NaN,Morris K. Udall
8,11746,NaN,NaN,1950-07-18,M,NaN,Mark Udall,U000038,7634,400412,...,12329,gIQAz6eM9O,Mark Udall,Mark,Udall,E.,NaN,Mark Udall,NaN,Thomas Udall
9,11746,NaN,NaN,1950-07-18,M,NaN,Mark Udall,U000038,7634,400412,...,12329,gIQAz6eM9O,Mark Udall,Mark,Udall,E.,NaN,Mark Udall,NaN,Stewart Lee Udall


In [210]:
help (get_list_obj)

Help on function get_list_obj in module norm:

get_list_obj(series)
    returns a dataseries of given series object. expands by the rows

